### Considering the column_diagnosis.arff dataset available at the course webpage’s homework tab. Using sklearn, apply a 10-fold stratified cross-validation with shuffling (random_state=0) for the assessment of predictive models along this section.

### 1) [3v] Compare the performance of 𝑘NN with 𝑘 = 5 and naïve Bayes with Gaussian assumption (consider all remaining parameters for each classifier as sklearn’s default):

### a. Plot two boxplots with the fold accuracies for each classifier.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold

from scipy.io.arff import loadarff
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Reading the ARFF file
data = loadarff('column_diagnosis.arff')
df = pd.DataFrame(data[0])
df['class'] = df['class'].str.decode('utf-8')

X = df.drop('class', axis=1)
y = df['class']

# kNN
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X, y)

# Bayesian
gnb = GaussianNB()
gnb.fit(X, y)


cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Cross-validation for kNN
knn_scores = cross_val_score(neigh, X, y, cv=cv)

# Cross-validation for Gaussian Naive Bayes
gnb_scores = cross_val_score(gnb, X, y, cv=cv)


# Boxplot 
data = pd.DataFrame({
    'label1': "Knn",
    'kNN_scores': knn_scores,
    'label2': "Gnb",
    'Gaussian_scores': gnb_scores
})

sns.boxplot(data=data)
plt.show()

### b. Using scipy, test the hypothesis “𝑘NN is statistically superior to naïve Bayes regarding accuracy”, asserting whether is true.

Para descobrir se kNN é estatisticamente superior, utilizamos um teste de hipóteses, considerando as seguintes hipóteses:

* H0: Accuracy do kNN igual à accuracy do GNB 
* H1: Accuracy do kNN maior do que a accuracy do GNB


In [ ]:
from scipy import stats

res = stats.ttest_rel(knn_scores, gnb_scores, alternative='greater')
print("knn > gnb ? p-value=",res.pvalue)

Baseado neste p-value, não podemos rejeitar a hipótese nula para níveis de significância 1%, 5% e 10%. 

Concluímos assim que não há uma diferença estatisticamente significativa entre os modelos kNN e naive Bayes e não podemos afirmar com confiança que kNN é estatisticamente superior ao naive Bayes em termos de accuracy.

### 2) [2.5v] Consider two 𝑘NN predictors with 𝑘 = 1 and 𝑘 = 5 (uniform weights, Euclidean distance, all remaining parameters as default). Plot the differences between the two cumulative confusion matrices of the predictors. Comment.

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Create models
knn1 = KNeighborsClassifier(n_neighbors=1, weights='uniform', metric='euclidean')

knn5 = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='euclidean')

# Initialize SKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Initialize confusion matrices arrays
confusion_matrices_knn1 = []
confusion_matrices_knn5 = []

# Cross-validation
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    knn1.fit(X_train, y_train)
    knn1_pred = knn1.predict(X_test)
    
    knn5.fit(X_train, y_train)
    knn5_pred = knn5.predict(X_test)
    
    # Add the values to the confusion matrix
    confusion_matrices_knn1.append(confusion_matrix(y_test, knn1_pred))
    confusion_matrices_knn5.append(confusion_matrix(y_test, knn5_pred))

# Calculate the final confusion matrices for each model
confusion_matrix_knn1 = np.sum(confusion_matrices_knn1, axis=0)
confusion_matrix_knn5 = np.sum(confusion_matrices_knn5, axis=0)

# Result confusion matrix
confusion_matrix = confusion_matrix_knn1 - confusion_matrix_knn5

# Plotting
labels = ['Hernia', 'Normal', 'Spondylolisthesis']
sns.heatmap(confusion_matrix, annot=True, fmt='g', xticklabels=labels, yticklabels=labels)

plt.title("Confusion Matrix Difference (KNN k=1 - KNN k=5)")
plt.xlabel("Predicted")
plt.ylabel("Real")
plt.show()


Este gráfico mostra a diferença entre as matrizes cumulativas dos preditores de kNN, com k=1 e k=5. Nas secções com valor positivo, o modelo com k=1 é superior, enquanto nos valores negativos k=5 é mais preciso. 
    
Observando o gráfico podemos ver que a matriz do preditor de KNN com k = 1, preveu mais casos errados em que a pessoa não tinha doenças e na realidade sofria de hérnia ou de spondylolisthesis. Por outro lado, também preveu mais vezes que o paciente tinha spondylolisthesis quando não apresentava doença. 
	
Podemos ver também que o preditor de KNN com k = 5, preveu mais casos em que a pessoa sofria de uma hérnia, que na realidade eram falsos positivos, visto que as pessoas não deviam ser diagnosticadas com hérnia.
	
Observando a diagonal,  que identifica os True Positives, podemos ver que o preditor de kNN com k=1 identifica mais casos de doenças corretamente,  que o preditor com k = 5. Podemos observar também que os dois preditores, foram idênticos nos casos em que se identificou uma doença e na realidade era outra (previu-se que o indivíduo sofria de hérnia mas na realidade sofria de spondylolisthesis, e vice-versa).
	
Concluindo, podemos ver que o preditor com k = 1 é superior em algumas secções, e que o preditor com k = 5 é mais preciso noutros casos. Deste modo, não podemos afirmar que um é universalmente melhor que o outro.

### 3) [1.5v] Considering the unique properties of column_diagnosis, identify three possible difficulties of naïve Bayes when learning from the given dataset.

* O dataset possui mais casos de Spondylolisthesis que do resto das classes (150 de Spondylolisthesis contra 60 casos de Hérnia e 100 casos normais). Uma vez que o naive Bayes assume que os parâmetros são condicionalmente independentes dentro de cada classe, uma classe dominante pode levar a que o naive Bayes tenha um "bias" e favoreça essa mesma classe incorretamente nas previsões.


* Naive Bayes assume que todos os parâmetros são independentes entre si. Em datasets clínicos como este, há uma grande possibilidade de alguns elementos estarem relacionados. Deste modo, ao ignorar relações entre parâmetros, o modelo pode produzir previsões imprecisas.


* O naive Bayes precisa de bastantes dados para aproximar com exatidão as suas funções de massa e densidade de probabilidade. Uma vez que o dataset tem uma dimensão média, de 310 elementos, a aproximação pode não ser a mais adequada, o que pode ser a causa da menor precisão deste modelo.